# Setting Up

In [2]:
%ls

 El volumen de la unidad C no tiene etiqueta.
 El n�mero de serie del volumen es: 5E35-A4EE

 Directorio de c:\Users\germa\Documents\Projects\Yolov11_ultralytics\ultralytics

13/10/2024  13:44    <DIR>          .
13/10/2024  13:44    <DIR>          ..
13/10/2024  00:26               784 __init__.py
13/10/2024  01:31    <DIR>          __pycache__
13/10/2024  00:26    <DIR>          assets
13/10/2024  01:31    <DIR>          cfg
13/10/2024  01:31    <DIR>          data
13/10/2024  01:42    <DIR>          datasets
13/10/2024  01:31    <DIR>          engine
13/10/2024  01:31    <DIR>          hub
13/10/2024  01:31    <DIR>          models
13/10/2024  01:31    <DIR>          nn
13/10/2024  00:26    <DIR>          solutions
13/10/2024  00:26    <DIR>          trackers
13/10/2024  01:31    <DIR>          utils
13/10/2024  13:38    <DIR>          weights
13/10/2024  13:44                 0 yolov11_tests.ipynb
               2 archivos            784 bytes
              15 dirs  155.372.777.472

In [ ]:
%pip install ultralytics

In [ ]:
%pwd

In [ ]:
# Update settings for project directories
from ultralytics import settings
settings.update({"datasets_dir": "c:\\Users\\germa\\Documents\\Projects\\Yolov11_ultralytics\\ultralytics\\datasets"})
settings.update({"weights_dir": "c:\\Users\\germa\\Documents\\Projects\\Yolov11_ultralytics\\ultralytics\\weights"})
settings.update({"runs_dir": "c:\\Users\\germa\\Documents\\Projects\\Yolov11_ultralytics\\ultralytics\\runs"})
print(settings)

In [ ]:
from ultralytics import YOLO

# Load a pretrained YOLO model (recommended for training)
model = YOLO("yolo11n-seg.pt")

# Train the model using the 'coco8.yaml' dataset for 3 epochs
results = model.train(
    task="segment",
    model=model
    data="coco8.yaml",
    epochs=5)